In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# Kodowanie słów i znaków metodą gorącej jedynki

Kodowanie metodą gorącej jedynki jest najpopularniejszym podstawowym sposobem zamieniania tokena w wektor. Z metody tej korzystaliśmy w początkowych przykładach przetwarzania zbiorów IMDB i Agencji Reutera przedstawionych w rozdziale 3. (używaliśmy jej do kodowania słów). Polega ona na przypisaniu do każdego słowa unikatowego indeksu będącego wartością całkowitoliczbową i umieszczoną w wektorze binarnym o długości N (rozmiar słownika). Wektor przyjmuje same wartości zerowe poza i-tym elementem, który przyjmuje wartość 1.

Oczywiście metoda ta może być również użyta na poziomie znaków. W celu wyjaśnienia praktycznej implementacji metody kodowania z gorącą jedynką chciałbym zaprezentować dwa przykłady: pierwszy z nich przedstawia kodowanie słów, a drugi kodowanie znaków.




Prosty przykład kodowania słów metodą gorącej jedynki:

In [3]:
import numpy as np

# Początkowa forma danych: jeden element na próbkę 
# (w tym przykładzie próbką jest zdanie, ale może ona być również całym dokumentem).
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# Zbuduj indeks wszystkich tokenów danych.
token_index = {}
for sample in samples:
    # Tokenizacja próbek poprzez metodę podziału. 
    # Podczas pracy z prawdziwymi danymi podziału dokonuje się również na znakach interpunkcyjnych i specjalnych.
    for word in sample.split():
        if word not in token_index:
            # Przypisywanie unikatowego indeksu do każdego unikatowego słowa.
            token_index[word] = len(token_index) + 1
            # Zwróć uwagę na to, że indeks 0 nie jest przypisywany do żadnego słowa.

# Wektoryzacja próbek. Bierzemy pod uwagę tylko max_length pierwszych słów każdej próbki.
max_length = 10

# Tu przechowujemy wyniki operacji:
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

Prosty przykład kodowania znaków metodą gorącej jedynki:

In [5]:
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  # Wszystkie znaki ASCII, które można wyświetlić.
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.

Pakiet Keras jest wyposażony w narzędzia przeznaczone do kodowania znaków i słów metodą gorącej jedynki (narzędzia te potrafią przetwarzać surowy tekst). W praktyce warto z nich korzystać, ponieważ wykonują wiele ważnych operacji, takich jak usuwanie znaków specjalnych i branie pod uwagę tylko N słów najczęściej występujących w zbiorze (zwykle stosuje się takie ograniczenie w celu uniknięcia pracy z bardzo dużymi przestrzeniami wektora wejściowego).

Przykład kodowania słów metodą gorącej jedynki przy użyciu gotowych narzędzi pakietu Keras:

In [7]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# Tworzy mechanizm tokenizacji skonfigurowany tak, 
# aby brał pod uwagę tylko 100 najczęściej występujących słów.
tokenizer = Tokenizer(num_words=1000)
# Buduje indeks słów.
tokenizer.fit_on_texts(samples)

# Zamienia łańcuchy na listy indeksów (wartości całkowitoliczbowe).
sequences = tokenizer.texts_to_sequences(samples)

# Możliwe jest również uzyskanie bezpośredniej binarnej reprezentacji kodowania metodą gorącej jedynki.
# Ten generator tokenów obsługuje także inne tryby wektoryzacji.
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# Przykład kodu pozwalającego na uzyskanie dostępu do indeksu słów.
word_index = tokenizer.word_index
print('Znaleziono %s unikatowych tokenów.' % len(word_index))

Found 9 unique tokens.



Odmianą kodowania metodą gorącej jedynki, która może zostać użyta, gdy liczba unikatowych tokenów w słowniku jest zbyt duża, aby obsłużyć ją w sposób jawny, jest sztuczka haszowania z gorącą jedynką (ang. one-hot hashing trick). Zamiast jawnie przypisywać indeks do każdego ze słów i utrzymywać odwołania do tych indeksów w słowniku, można haszować słowa do formy wektorów o określonym rozmiarze. Zwykle robi się to za pomocą bardzo lekkiej funkcji haszującej. Główną zaletą tej metody jest brak konieczności utrzymywania jawnego indeksu słów, co pozwala oszczędzić przestrzeń pamięci i zakodować dane w locie (możliwe jest natychmiastowe wygenerowanie wektorów tokenu, bez potrzeby przyglądania się całości dostępnych danych). Wadą tego rozwiązania jest możliwość wystąpienia konfliktów haszy (ang. hash collisions), polegających na przepisaniu tego samego hasza dwóm różnym słowom (w takim przypadku żaden model uczenia maszynowego analizujący uzyskane hasze nie będzie mógł odróżnić od siebie tych słów). Prawdopodobieństwo wystąpienia konfliktów haszy maleje, gdy przestrzeń haszowania jest o wiele większa od całkowitej liczby unikatowych haszowanych tokenów.

Prosty przykład sztuczki haszowania słów metodą gorącej jedynki:

In [9]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# Słowa są zapisywane w postaci wektorów o długości 1000. 
# Jeżeli przetworzymy przykład, w którym znajduje się około 1000 różnych słów,
# to zauważymy wiele konfliktów haszy, 
# które doprowadzą do pogorszenia dokładności tej metody kodowania.
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # Słowom przypisywane są losowe wartości całkowite indeksu z zakresu od 0 do 1000.        
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.